# Register the ML Model to an Azure ML Workspace

In this notebookw, we will register the trained machine learning model to an Azure Machine Learning Workspace. 

Azure ML provides a seamless way to register the model's artifacts to Azure ML for deployment and further management. This ensures reproducibility, versioning, and easy access to the model for future use.


## Load the model and artifacts

In [ ]:
import pickle

# Load the model
with open('Users/lubraz/reasoning-over-ml/train/model_artifacts/model.pkl', 'rb') as f:
    model = pickle.load(f)

# Load a sample of X_test in order to get the dtypes and use it for inference
with open('Users/lubraz/reasoning-over-ml/train/model_artifacts/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

## Configure AML client

In [134]:
import os
from dotenv import load_dotenv
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

load_dotenv()

subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID")
resource_group = os.environ.get("AZURE_RESOURCE_GROUP")
workspace = os.environ.get("WORKSPACE_NAME")

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

print("MLClient loaded successfully:", ml_client.workspace_name)

MLClient loaded successfully: wkmllab02


## Register the model to AML Workspace

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="Users/lubraz/reasoning-over-ml/train/model_artifacts/",
    type=AssetTypes.CUSTOM_MODEL,
    name="store-sales-forecasting",
    description="Model for store sales forecasting. Trained on the Store Sales - Time Series Forecasting dataset from Kaggle.",
    tags={"model_type": "regression", "dataset": "store-sales-forecasting"},
)

ml_client.models.create_or_update(file_model)

Uploading model_artifacts (50.79 MBs): 100%|██████████| 50791302/50791302 [00:01<00:00, 46404330.63it/s]




Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'store-sales-forecasting', 'description': 'Model for store sales forecasting. Trained on the Store Sales - Time Series Forecasting dataset from Kaggle.', 'tags': {'model_type': 'regression', 'dataset': 'store-sales-forecasting'}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/2168a4ef-0538-435e-9ae6-beb6183f2769/resourceGroups/RG-ML/providers/Microsoft.MachineLearningServices/workspaces/wkmllab02/models/store-sales-forecasting/versions/4', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/lubraz5/code', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fb17f80fc10>, 'serialize': <msrest.serialization.Serializer object at 0x7fb17f80cf40>, 'version': '4', 'latest_version': None, 'path': 'azureml://subscriptions/2168a4ef-0538-435e-9ae6-beb6183f2769/resource

In [5]:
import mlflow
from mlflow.models.signature import infer_signature

signature = infer_signature(X_test, predictions)

# Set the new experiment (if it doesn't exist, MLflow will create it)
experiment_name = "Store-Sales-Forecasting-Experiment"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name="XGBoost Model Run"):
    mlflow.log_metric("RMSLE", results['RMSLE'][0])
    mlflow.log_metric("RMSE", results['RMSE'][0])
    mlflow.log_metric("MSE", results['MSE'][0])
    mlflow.log_metric("MAE", results['MAE'][0])
    mlflow.log_metric("MAE_percent", results['MAE_percent'][0])
    

    mlflow.xgboost.log_model(model, artifact_path="xgb_model", signature=signature)

    
    print("XGBoost model and metrics successfully logged under experiment:", experiment_name)

KeyboardInterrupt: 